# 2. Preprocesamiento de datos

En este paso crucial, preparamos nuestro conjunto de datos para la fase de modelado, asegurándonos de que nuestros datos estén en el formato apropiado para los algoritmos de aprendizaje automático. Las principales tareas en el preprocesamiento de datos incluyen codificar variables categóricas, escalar características numéricas y dividir los datos en conjuntos de entrenamiento y prueba. Aquí hay un desglose de cada tarea:

### Codificación de variables categóricas

Muchos modelos de aprendizaje automático requieren entrada numérica, por lo que las variables categóricas deben convertirse a un formato numérico. Usamos codificación one-hot para transformar las características categóricas ('Sex', 'BP', 'Cholesterol') en vectores binarios, que representan la presencia o ausencia de cada categoría con 0 y 1.

### Dividir el conjunto de datos

Dividimos nuestro conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba, lo que nos permite entrenar nuestros modelos en un subconjunto de datos y evaluar su desempeño en un conjunto independiente. Normalmente, asignamos el 80% de los datos para entrenamiento y el 20% para pruebas. Esta división garantiza que nuestro modelo pueda generalizarse bien a datos invisibles.

### Escalado de funciones numéricas

Finalmente, para garantizar que cada característica numérica contribuya por igual a la predicción del modelo, escalamos las características ('Age', 'Na', 'K'). Se aplica una escala estándar para normalizar los datos, estableciendo la media en 0 y la desviación estándar en 1. Este paso ayuda a mejorar la convergencia del modelo y su sensibilidad a los valores de las características.


Al final de estos pasos de preprocesamiento, nuestros datos están limpios, formateados y listos para la etapa de modelado.


In [2]:
#Libraries needed for EDA only
# to handle datasets
import pandas as pd
import numpy as np
import seaborn as sns

# for plotting
import matplotlib.pyplot as plt
# Set the aesthetic style of the plots
sns.set_theme(context='notebook', style='darkgrid', palette='deep', font='sans-serif', font_scale=1, color_codes=True, rc=None)

# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)

import statsmodels.api as sm

# for the yeo-johnson transformation
import scipy.stats as stats

# to divide train and test set
from sklearn.model_selection import train_test_split

# feature scaling
from sklearn.preprocessing import MinMaxScaler

# to save the trained scaler class
import joblib

In [3]:
# Load the dataset
data_path = 'Drug.csv'
data = pd.read_csv(data_path)


In [22]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

# Defining the feature set and the target variable
X = data.drop('Drug', axis=1)
y = data['Drug']

# Identifying numerical and categorical features
numerical_features = ['Age', 'Na', 'K']
categorical_features = ['Sex']
ordinal_features = ['BP', 'Cholesterol']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Defining a column transformer with one-hot encoding for categorical features and scaling for numerical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features),
        ('ord', OrdinalEncoder(), ordinal_features)
    ])

# Fitting the transformer on the training data and transforming both the training and test data
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

print("X train processed: \n" , X_train_processed)
print(X_train_processed.shape)
print("X test processed: \n" ,X_test_processed)
print(X_test_processed.shape)

X train processed: 
 [[ 0.49932185  0.52558168  0.61844831 ...  0.          0.
   1.        ]
 [ 1.48092384  1.27696108  1.36539494 ...  1.          1.
   1.        ]
 [ 0.99012284 -1.50265424  1.20309138 ...  0.          1.
   1.        ]
 ...
 [-0.97308114  1.35356617 -1.17559073 ...  0.          0.
   0.        ]
 [ 1.35822359  0.73394818 -0.02969257 ...  0.          2.
   0.        ]
 [-1.03443126 -0.36541914  0.0031297  ...  0.          1.
   0.        ]]
(180, 7)
X test processed: 
 [[-0.54363027 -1.46208406 -0.21205104  0.          1.          1.
   1.        ]
 [-1.77063276  1.14814076  0.2146949   1.          0.          0.
   1.        ]
 [-1.64793251 -1.24114328  0.75267499  1.          0.          2.
   1.        ]
 [ 0.86742259 -0.50042857  0.64279667  1.          0.          1.
   0.        ]
 [ 0.1312211   1.26657771 -1.39353961  0.          1.          1.
   1.        ]
 [ 0.3766216   1.18780729 -0.23221894  0.          1.          0.
   0.        ]
 [-1.64793251 -1.145

# 3. Construcción de modelos

En esta fase, construiremos y entrenaremos cuatro modelos distintos de aprendizaje automático utilizando los algoritmos que hemos seleccionado: K-vecinos más cercanos (KNN), regresión logística, árboles de decisión y máquinas de vectores de soporte (SVM). Cada modelo se entrenará en nuestro conjunto de datos de entrenamiento y luego se evaluará en función de su rendimiento en el conjunto de datos de prueba. Los pasos para la construcción del modelo son los siguientes:

### K-Nearest Neighbors (KNN)
KNN hace predicciones para un nuevo punto de datos basándose en la etiqueta de mayoría entre sus 'k' vecinos más cercanos en el espacio de características. Es un algoritmo sencillo pero potente, particularmente eficaz para conjuntos de datos donde instancias similares tienen etiquetas similares.

### Logistic Regression
La regresión logística es un modelo estadístico que en su forma básica utiliza una función logística para modelar una variable dependiente binaria, aunque en formas más complejas se puede utilizar para modelar resultados multiclase. Es ampliamente utilizado para tareas de clasificación binaria.

### Decision Trees
Los árboles de decisión son estructuras similares a diagramas de flujo que utilizan métodos de ramificación para ilustrar todos los resultados posibles de una decisión. Son fáciles de entender e interpretar, pero pueden ser propensos a sobreajustarse, especialmente con conjuntos de datos complejos.

### Support Vector Machines (SVM)
Las SVM son un conjunto de métodos de aprendizaje supervisados que se utilizan para clasificación, regresión y detección de valores atípicos. Un modelo SVM es una representación de los ejemplos como puntos en el espacio, mapeados de modo que los ejemplos de las categorías separadas estén divididos por una brecha clara que sea lo más amplia posible.

Después de entrenar cada modelo, evaluaremos su desempeño con los datos de prueba para comprender su efectividad. La métrica de precisión nos ayudará a determinar cuál es el modelo de mejor rendimiento entre los que hemos entrenado.


In [23]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Initialize the models
knn = KNeighborsClassifier()
log_reg = LogisticRegression(max_iter=1000)  # Increased max_iter for convergence
decision_tree = DecisionTreeClassifier()
svm = SVC()

# Dictionary to store models and their respective accuracies
models = {
    'K-Nearest Neighbors': knn,
    'Logistic Regression': log_reg,
    'Decision Tree': decision_tree,
    'Support Vector Machine': svm
}

# Dictionary to store accuracies
accuracies = {}

# Train and evaluate each model
for model_name, model in models.items():
    # Train the model
    model.fit(X_train_processed, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test_processed)

    # Calculate the accuracy
    accuracy = accuracy_score(y_test, y_pred)
    accuracies[model_name] = accuracy

accuracies



{'K-Nearest Neighbors': 0.85,
 'Logistic Regression': 0.95,
 'Decision Tree': 0.75,
 'Support Vector Machine': 0.95}

In [ ]:

dataTrain= pd.concat([pd.DataFrame(X_train_processed), y_train], axis=1)
print(dataTrain)
dataTest= pd.concat([pd.DataFrame(X_test_processed), y_test], axis=1)
print(y_test)
print("dataTest")
print(dataTest)

In [ ]:
from pycaret.classification import *
# init setup on exp
s= setup(data, target = 'Drug', session_id = 123)

#compare models using OOP
bestPycaret = compare_models(n_select = 1)
predPycaret= predict_model(bestPycaret)

In [1]:
from pycaret.classification import load_model
savedLGBM = load_model('BestLGBMModelEDA')
savedLGBM

Transformation Pipeline and Model Successfully Loaded


Pipeline(memory=Memory(location=None),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(transformer=LabelEncoder())),
                ('numerical_imputer',
                 TransformerWrapper(include=['Age', 'Na', 'K'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=['Sex', 'BP', 'Cholesterol'],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('...
                                                               mapping=[{'col': 'Sex',
                                                                         'data_type': dtype('O'),
                                                                         'mapping': F      0
M      1
NaN   -1
dtype: int64},
                                                                        {'col': 'Cholesterol',
                                                                         'data_type': dtype('O'),
                                                                         'mapping': HIGH      0
NORMAL    1
NaN      -1
dtype: int64}]))),
                ('onehot_encoding',
                 TransformerWrapper(include=['BP'],
                                    transformer=OneHotEncoder(cols=['BP'],
                                                              handle_missing='return_nan',
                                                              use_cat_names=True))),
                ('trained_model', LGBMClassifier(n_jobs=-1, random_state=123))])

In [25]:
from sklearn.metrics import confusion_matrix, classification_report
from tabulate import tabulate
# Initializing a dictionary to store evaluation metrics
model_evaluation_metrics = {}

# Iterating over each model to evaluate
for model_name, model in models.items():
    # Predict on the test set
    y_pred = model.predict(X_test_processed)

    # Calculate confusion matrix and classification report
    conf_matrix = confusion_matrix(y_test, y_pred)
    classif_report = classification_report(y_test, y_pred)

    # Storing the results
    model_evaluation_metrics[model_name] = {
        'Confusion Matrix': conf_matrix,
        'Classification Report': classif_report
    }

    # Outputting the results for review
    print(f"{model_name} Evaluation Metrics:")
    print("Confusion Matrix:")
    print(tabulate(conf_matrix, headers=model.classes_, tablefmt='grid'))
    print("\nClassification Report:")
    print(classif_report)
    print("\n" + "="*60 + "\n")

K-Nearest Neighbors Evaluation Metrics:
Confusion Matrix:
+---------+---------+---------+---------+---------+
|   drugA |   drugB |   drugC |   drugX |   drugY |
+=========+=========+=========+=========+=========+
|       2 |       0 |       0 |       0 |       1 |
+---------+---------+---------+---------+---------+
|       0 |       2 |       0 |       0 |       0 |
+---------+---------+---------+---------+---------+
|       0 |       0 |       0 |       2 |       0 |
+---------+---------+---------+---------+---------+
|       0 |       0 |       0 |       6 |       0 |
+---------+---------+---------+---------+---------+
|       0 |       0 |       0 |       0 |       7 |
+---------+---------+---------+---------+---------+

Classification Report:
              precision    recall  f1-score   support

       drugA       1.00      0.67      0.80         3
       drugB       1.00      1.00      1.00         2
       drugC       0.00      0.00      0.00         2
       drugX       0.75  

/opt/anaconda3/envs/drugs_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/drugs_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/drugs_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu